# 암 데이터 Cox Lasso 생존 분석 파이썬 코드 예제
* Cox Lasso 생존 분석은 생존 시간 데이터에 Lasso 규제를 적용하여 변수 선택과 모델 학습을 동시에 수행하는 방법입
* 암 데이터와 같은 생존 시간 데이터 분석에 유용
* 파이썬에서는 lifelines 패키지를 활용하여 쉽게 구현

1. 라이브러리 설치 및 데이터 준비

In [ ]:
import pandas as pd
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기 (예시: CSV 파일)
data = pd.read_csv("cancer_data.csv")

# 생존 시간, 사건 발생 여부, 그리고 다양한 변수 포함
# 예시: time (생존 시간), event (사건 발생 여부: 1=사망, 0=생존), age, gender, stage 등

# 데이터 분할 (훈련 데이터와 테스트 데이터)
X = data.drop(['time', 'event'], axis=1)  # 독립 변수
y = data[['time', 'event']]  # 종속 변수 (생존 시간, 사건 발생 여부)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 표준화 (필요한 경우)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

2. Cox Lasso 모델 학습

In [ ]:
# Grid Search를 이용한 하이퍼파라미터 튜닝

from sklearn.model_selection import GridSearchCV
from lifelines import CoxPHFitter

# 하이퍼파라미터 탐색 범위 설정
param_grid = {'penalizer': [0.1, 1, 10]}

# CoxPHFitter 모델 생성
cph = CoxPHFitter()

# GridSearchCV 객체 생성
grid_search = GridSearchCV(estimator=cph, param_grid=param_grid, cv=5, scoring='neg_log_loss')

# Grid Search 실행
grid_search.fit(X, duration_col='time', event_col='event')

# 최적의 파라미터 출력
print(grid_search.best_params_)

* param_grid: 탐색할 하이퍼파라미터와 값들의 딕셔너리.
* GridSearchCV: Scikit-learn의 GridSearchCV 클래스를 이용하여 Grid Search를 수행
* cv: 교차 검증 횟수를 설정
* scoring: 모델 평가에 사용할 지표를 설정. 
  * neg_log_loss는 일반적으로 분류 문제에서 사용되는 지표이지만, 생존 분석에서도 사용될 수 있다. 
  * 생존 분석에서는 concordance index를 사용하는 것이 더 적절할 수 있다.
  
### 주의 사항
* 계산 비용: 하이퍼파라미터 조합이 많아질수록 계산 시간이 오래 걸릴 수 있다.
* 과적합: Grid Search를 통해 찾은 최적의 하이퍼파라미터 조합이 항상 일반화 성능이 가장 좋은 것은 아니다.
* 평가 지표: 사용하는 평가 지표에 따라 최적의 하이퍼파라미터 조합이 달라질 수 있다.

### 추가 고려 사항
* Randomized Search: Grid Search보다 더 효율적으로 하이퍼파라미터를 탐색할 수 있는 Randomized Search를 사용할 수 있다.
* Bayesian Optimization: Bayesian Optimization은 이전 결과를 기반으로 다음 탐색 지점을 선택하여 더 효율적으로 최적의 하이퍼파라미터를 찾을 수 있다.
* Early Stopping: 학습 중간에 성능이 더 이상 향상되지 않으면 학습을 중단하여 시간을 절약할 수 있다.

In [ ]:
cph = CoxPHFitter(penalizer=1.0)  # penalizer 값 조절 (Lasso 규제 강도)
cph.fit(X_train, duration_col='time', event_col='event')

# 모델 요약
cph.print_summary()

3-1. 모델 평가

In [ ]:
# Concordance index (C-index) 계산
#c_index = concordance_index(y_test['time'], -cph.predict_partial_hazard(X_test), y_test['event'])
#print("Concordance index:", c_index)

# 예측
#predicted_risk = cph.predict_partial_hazard(X_test)

In [ ]:
3-2. K-Fold 교차 검증

In [ ]:
# 각 fold에서 모델 학습 및 평가
c_indices = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    cph = CoxPHFitter(penalizer=1.0)  # penalizer 값 조절
    cph.fit(X_train, duration_col='time', event_col='event')

    # Concordance index 계산
    c_index = concordance_index(y_test['time'], -cph.predict_partial_hazard(X_test), y_test['event'])
    c_indices.append(c_index)

# 평균 Concordance index 계산
print("Mean Concordance index:", np.mean(c_indices))

## 코드 설명
* penalizer: Lasso 규제의 강도를 조절하는 파라미터. 값이 클수록 더 많은 변수가 제외. 적절한 값은 교차 검증을 통해 찾을 수 있다.
* Concordance index: 생존 분석 모델의 성능을 평가하는 지표. 값이 0.5에 가까울수록 예측 성능이 낮고, 1에 가까울수록 예측 성능이 높다.
* 변수 선택: cph.summary()를 통해 각 변수의 유의성을 확인하고, 필요에 따라 변수를 추가하거나 제거할 수 있다.
* 교차 검증: 모델의 일반화 성능을 평가하기 위해 교차 검증을 수행하는 것이 좋다.
* 시간 의존적 공변량: 시간에 따라 변하는 공변량이 있다면, time_dependent_covariates 인자를 사용하여 모델에 포함할 수 있다.

# 암 데이터 Cox Ridge 생존 분석 파이썬 코드 예제
* Cox Ridge 모델은 Cox 비례 위험 모델에 Ridge 규제를 적용하여 다중공선성 문제를 해결하고 모델의 안정성을 높이는 방법
* Ridge 규제는 모델의 복잡도를 낮춰 과적합을 방지하는 효과가 있다.

1. 라이브러리 설치 및 데이터 준비

In [ ]:
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기 (예시: CSV 파일)
data = pd.read_csv("cancer_data.csv")

# 생존 시간, 사건 발생 여부, 그리고 다양한 변수 포함
# 예시: time (생존 시간), event (사건 발생 여부: 1=사망, 0=생존), age, gender, stage 등

# 데이터 분할 (훈련 데이터와 테스트 데이터)
X = data.drop(['time', 'event'], axis=1)  # 독립 변수
y = data[['time', 'event']]  # 종속 변수 (생존 시간, 사건 발생 여부)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 표준화 (필요한 경우)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

2. Cox Ridge 모델 생성 및 학습

In [ ]:
cph = CoxPHFitter(penalizer=1.0)  # penalizer 값 조절 (Ridge 규제 강도)
cph.fit(X_train, duration_col='time', event_col='event')

# 모델 요약
cph.print_summary()

3-1. 모델 평가

In [ ]:
# Concordance index (C-index) 계산
#c_index = concordance_index(y_test['time'], -cph.predict_partial_hazard(X_test), y_test['event'])
#print("Concordance index:", param_grid: 탐색할 하이퍼파라미터와 값들의 딕셔너리입니다.

# 예측
#predicted_risk = cph.predict_partial_hazard(X_test)

3-2. K-Fold 교차 검증

In [ ]:
# 각 fold에서 모델 학습 및 평가
c_indices = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    cph = CoxPHFitter(penalizer=1.0)  # penalizer 값 조절
    cph.fit(X_train, duration_col='time', event_col='event')

    # Concordance index 계산
    c_index = concordance_index(y_test['time'], -cph.predict_partial_hazard(X_test), y_test['event'])
    c_indices.append(c_index)

# 평균 Concordance index 계산
print("Mean Concordance index:", np.mean(c_indices))

## 코드 설명
* penalizer: Ridge 규제의 강도를 조절하는 파라미터. 값이 클수록 규제가 강해지고, 모델의 복잡도가 낮아진다. 적절한 값은 교차 검증을 통해 찾을 수 있다.
* Concordance index: 생존 분석 모델의 성능을 평가하는 지표. 값이 0.5에 가까울수록 예측 성능이 낮고, 1에 가까울수록 예측 성능이 높다.
* 변수 선택: cph.summary()를 통해 각 변수의 유의성을 확인할 수 있다.
* 교차 검증: 모델의 일반화 성능을 평가하기 위해 교차 검증을 수행하는 것이 좋다.
시간 의존적 공변량: 시간에 따라 변하는 공변량이 있다면, time_dependent_covariates 인자를 사용하여 모델에 포함할 수 있습니다.
다른 규제: Ridge 외에도 Lasso, Elastic Net 등 다른 규제를 사용할 수 있습니다.
Cox Ridge vs. Cox Lasso
Ridge: 모든 변수에 규제를 적용하여 다중공선성 문제를 해결하고 모델의 안정성을 높이는 데 효과적입니다.
Lasso: 불필요한 변수를 제거하여 모델을 간결하게 만들고, 변수 선택에도 활용될 수 있습니다.